In [1]:
%%time
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from statistics import mean 
import time
import torch

import sys
script_path = os.path.abspath("pyscripts")
if script_path not in sys.path:
    sys.path.append(script_path)
from utils import *
from load_data import *
from lstm import *

CPU times: user 2.75 s, sys: 2.27 s, total: 5.02 s
Wall time: 1.3 s


In [2]:
# time sequence
train_window = 48
input_size = 1
output_size = 24
hidden_size = 48
num_layers = 2
learning_rate = 0.005

# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
obs = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/Geotab_Obs_Ratio05.npy')
# swap rows and columns, obs = np.swapaxes(obs, 0, 1)
obs = obs.T
times = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/Geotab_Obs_TS_Ratio05.npy')
times = [datetime.utcfromtimestamp(x.tolist()/1e9) for x in times]
stations = pd.read_csv('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/selected_Geotab_stations_Ratio05.csv')
stations = stations[stations.columns[[3,1,2,4]]]

weather_station_2015_2020 = pd.read_csv("/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/weather_station_2015_2020.csv")
weather_time_2015_2020 = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/weather_time_2015_2020.npy', allow_pickle=True)

weather_obs_2015_2020 = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/weather_obs_2015_2020.npy')
weather_obs_2010_2020 = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/weather_obs_2010_2020.npy')

In [4]:
stations5 = set(stations.sort_values('missing')[:455]['geohash'])
stations10= set(stations.sort_values('missing')[:1650]['geohash'])
stations15= set(stations.sort_values('missing')[:2107]['geohash'])
stations20= set(stations.sort_values('missing')[:2562]['geohash'])
wu_stations = set(weather_station_2015_2020['station'])

In [5]:
num_epochs = 101
epoch_interval = 2

In [6]:
# two dataset used different norm_min, norm_max
# read save data
loaded5 = torch.load('./checkpoint/IoT_final/TensorData/Torch_db5WU')
train_tensors5 = loaded5['train']
test_tensors5 = loaded5['test']
test_times5 = loaded5['selected_days']
norm_min5, norm_max5 = train_tensors5.min, train_tensors5.max
print(norm_min5, norm_max5)

# read save data
loaded10 = torch.load('./checkpoint/IoT_final/TensorData/Torch_db10WU')
train_tensors10 = loaded10['train']
test_tensors10 = loaded10['test']
test_times10 = loaded10['selected_days']
norm_min10, norm_max10 = train_tensors10.min, train_tensors10.max
print(norm_min10, norm_max10)

-22.55 48.0
-29.632716049382715 48.0


In [7]:
checkpoint5 = torch.load('./checkpoint/IoT_final/checkpoint-5.5-{epoch}.pth.tar'.format(epoch=60))
checkpoint10 = torch.load('./checkpoint/IoT_final/checkpoint-10-{epoch}.pth.tar'.format(epoch=23))
checkpoint15 = torch.load('./checkpoint/IoT_final/checkpoint-15-{epoch}.pth.tar'.format(epoch=8))
checkpoint20 = torch.load('./checkpoint/IoT_final/checkpoint-20-{epoch}.pth.tar'.format(epoch=7))

checkpoint5IoT5WU = torch.load('./checkpoint/IoT_final/checkpoint-5.5-IoT-5-W-{epoch}.pth.tar'.format(epoch=18))
checkpoint10IoT5WU = torch.load('./checkpoint/IoT_final/checkpoint-10-IoT-5-W-{epoch}.pth.tar'.format(epoch=9))
checkpoint15IoT5WU = torch.load('./checkpoint/IoT_final/checkpoint-15-IoT-5-W-{epoch}.pth.tar'.format(epoch=5))
checkpoint20IoT5WU = torch.load('./checkpoint/IoT_final/checkpoint-20-IoT-5-W-{epoch}.pth.tar'.format(epoch=7))

checkpoint5IoT10WU = torch.load('./checkpoint/IoT_final/checkpoint-5.5-IoT-10-W-{epoch}.pth.tar'.format(epoch=15))
checkpoint10IoT10WU = torch.load('./checkpoint/IoT_final/checkpoint-10-IoT-10-W-{epoch}.pth.tar'.format(epoch=12))
checkpoint15IoT10WU = torch.load('./checkpoint/IoT_final/checkpoint-15-IoT-10-W-{epoch}.pth.tar'.format(epoch=5))
checkpoint20IoT10WU = torch.load('./checkpoint/IoT_final/checkpoint-20-IoT-10-W-{epoch}.pth.tar'.format(epoch=3))


_, model5, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model10, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model15, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model20, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)

_, model5IoT5WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model10IoT5WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model15IoT5WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model20IoT5WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)

_, model5IoT10WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model10IoT10WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model15IoT10WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model20IoT10WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)


model5.load_state_dict(checkpoint5['bestLossModel'])
model10.load_state_dict(checkpoint10['bestLossModel'])
model15.load_state_dict(checkpoint15['bestLossModel'])
model20.load_state_dict(checkpoint20['bestLossModel'])

model5IoT5WU.load_state_dict(checkpoint5IoT5WU['bestLossModel'])
model10IoT5WU.load_state_dict(checkpoint10IoT5WU['bestLossModel'])
model15IoT5WU.load_state_dict(checkpoint15IoT5WU['bestLossModel'])
model20IoT5WU.load_state_dict(checkpoint20IoT5WU['bestLossModel'])

model5IoT10WU.load_state_dict(checkpoint5IoT10WU['bestLossModel'])
model10IoT10WU.load_state_dict(checkpoint10IoT10WU['bestLossModel'])
model15IoT10WU.load_state_dict(checkpoint15IoT10WU['bestLossModel'])
model20IoT10WU.load_state_dict(checkpoint20IoT10WU['bestLossModel'])

<All keys matched successfully>

In [8]:
test_pred_orig_dict5 = predict(model5, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)
test_pred_orig_dict10 = predict(model10, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)
test_pred_orig_dict15 = predict(model15, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)
test_pred_orig_dict20 = predict(model20, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)

test_pred_orig_dict5_5WU = predict(model5IoT5WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)
test_pred_orig_dict10_5WU = predict(model10IoT5WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)
test_pred_orig_dict15_5WU = predict(model15IoT5WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)
test_pred_orig_dict20_5WU = predict(model20IoT5WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)

test_pred_orig_dict5_10WU = predict(model5IoT10WU, test_tensors10, (norm_min10, norm_max10), device, stationlist = stations5)
test_pred_orig_dict10_10WU = predict(model10IoT10WU, test_tensors10, (norm_min10, norm_max10), device, stationlist = stations5)
test_pred_orig_dict15_10WU = predict(model15IoT10WU, test_tensors10, (norm_min10, norm_max10), device, stationlist = stations5)
test_pred_orig_dict20_10WU = predict(model20IoT10WU, test_tensors10, (norm_min10, norm_max10), device, stationlist = stations5)

# Here, let's try to use a model to predict same observations normalized in other values different from train_tensors
# test_pred_orig_dict5_10WU = predict(model5IoT10WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)
# test_pred_orig_dict10_10WU = predict(model10IoT10WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)
# test_pred_orig_dict15_10WU = predict(model15IoT10WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)
# test_pred_orig_dict20_10WU = predict(model20IoT10WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations5)

##################################
_,_,_,_ = stat_scores(test_pred_orig_dict5)
_,_,_,_ = stat_scores(test_pred_orig_dict10)
_,_,_,_ = stat_scores(test_pred_orig_dict15)
_,_,_,_ = stat_scores(test_pred_orig_dict20)
print('\n')
_,_,_,_ = stat_scores(test_pred_orig_dict5_5WU)
_,_,_,_ = stat_scores(test_pred_orig_dict10_5WU)
_,_,_,_ = stat_scores(test_pred_orig_dict15_5WU)
_,_,_,_ = stat_scores(test_pred_orig_dict20_5WU)
print('\n')
_,_,_,_ = stat_scores(test_pred_orig_dict5_10WU)
_,_,_,_ = stat_scores(test_pred_orig_dict10_10WU)
_,_,_,_ = stat_scores(test_pred_orig_dict15_10WU)
_,_,_,_ = stat_scores(test_pred_orig_dict20_10WU)

Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (3.4387644436254536, 2.3407304286956787), (3.876985102985248, 2.670888900756836), (4.310313237041598, 2.949211597442627)
Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (3.412777726606036, 2.409334421157837), (3.77913083049322, 2.7551872730255127), (4.33056241477329, 3.220960855484009)
Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (3.2225732411184325, 2.3598413467407227), (3.6363234095082353, 2.628476858139038), (4.105673163322117, 2.971999168395996)
Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (3.2506330313763954, 2.4456987380981445), (3.839813822166878, 2.842639207839966), (4.319781803695989, 3.079951047897339)


Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (2.606095680406721, 1.842820167541504), (3.18259905856489, 2.2648518085479736), (3.9106587669179182, 2.700509548187256)
Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (2.5973128701323653, 1.9310272

In [9]:
test_pred_orig_dict10 = predict(model10, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations10)
test_pred_orig_dict10_5WU = predict(model10IoT5WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations10)
test_pred_orig_dict10_10WU = predict(model10IoT10WU, test_tensors10, (norm_min10, norm_max10), device, stationlist = stations10)

test_pred_orig_dict15 = predict(model15, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations15)
test_pred_orig_dict15_5WU = predict(model15IoT5WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations15)
test_pred_orig_dict15_10WU = predict(model15IoT10WU, test_tensors10, (norm_min10, norm_max10), device, stationlist = stations15)

test_pred_orig_dict20 = predict(model20, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations20)
test_pred_orig_dict20_5WU = predict(model20IoT5WU, test_tensors5, (norm_min5, norm_max5), device, stationlist = stations20)
test_pred_orig_dict20_10WU = predict(model20IoT10WU, test_tensors10, (norm_min10, norm_max10), device, stationlist = stations20)

##############################
_,_,_,_ = stat_scores(test_pred_orig_dict10)
_,_,_,_ = stat_scores(test_pred_orig_dict10_5WU)
_,_,_,_ = stat_scores(test_pred_orig_dict10_10WU)
print('\n')
_,_,_,_ = stat_scores(test_pred_orig_dict15)
_,_,_,_= stat_scores(test_pred_orig_dict15_5WU)
_,_,_,_ = stat_scores(test_pred_orig_dict10_10WU)
print('\n')
_,_,_,_= stat_scores(test_pred_orig_dict20)
_,_,_,_= stat_scores(test_pred_orig_dict20_5WU)
_,_,_,_ = stat_scores(test_pred_orig_dict10_10WU)

Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (2.974440732046452, 2.249952793121338), (3.8249538443778763, 2.7707934379577637), (4.33056241477329, 3.220960855484009)
Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (2.585607501868186, 1.876233696937561), (3.097661483779469, 2.2609384059906006), (3.6585640390653125, 2.623060464859009)
Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (2.9121404859798585, 2.116349935531616), (3.514823720535354, 2.5380074977874756), (3.8934851248645477, 2.842907428741455)


Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (3.078525730131477, 2.3276076316833496), (3.7114140575993138, 2.7115671634674072), (4.407029103327788, 3.298968553543091)
Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (2.7619946462039313, 2.0050010681152344), (3.337225564824772, 2.434685707092285), (4.157285632985604, 2.8132243156433105)
Minimum, Mean and Maximum RMSE/MAE for Testing Stations are: (2.9121404859798585, 2.11